In [1]:
from deap import base
from deap import creator
from deap import tools
from deap import algorithms

import csv
import array
import random
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
class TravelingSalesmanProblem:
    def __init__(self, name):
        self.name = name
        self.locations = []
        self.distances = []
        self.tspSize = 0
        self.__initData()

    def __len__(self):
        return self.tspSize
    
    def __initData(self):
        self.__createData()
        self.tspSize = len(self.locations)

    def __createData(self):
        self.locations = []

        with open(self.name + ".tsp") as f:
            reader = csv.reader(f, delimiter=" ")
            for i, row in enumerate(reader):
                if i >= 50:  # Limit to the first 50 cities
                    break
                row1 = (float(row[0]), float(row[2]))
                self.locations.append(np.asarray(row1))

            self.tspSize = len(self.locations)
            # print("length = {}, locations = {}".format(self.tspSize, self.locations))
            self.distances = [[0] * self.tspSize for _ in range(self.tspSize)]

            for i in range(self.tspSize):
                for j in range(i + 1, self.tspSize):
                    distance = np.linalg.norm(self.locations[j] - self.locations[i])
                    self.distances[i][j] = distance
                    self.distances[j][i] = distance
                    # print("{}, {}: location1 = {}, location2 = {} => distance = {}".format(i, j, self.locations[i], self.locations[j], distance))

    def getTotalDistance(self, indices):
        distance = self.distances[indices[-1]][indices[0]]

        for i in range(len(indices) - 1):
            distance += self.distances[indices[i]][indices[i+1]]

        return distance
    
    def plotData(self, indices):
        plt.scatter(*zip(*self.locations), marker='.', color='red')
        locs = [self.locations[i] for i in indices]
        locs.append(locs[0])
        
        # Debugging: Print the locations
        # print("Plotted locations:", locs)

        # Optional: Highlight start and end points
        plt.plot(*zip(*locs), linestyle='-', color='blue')
        plt.scatter(*locs[0], color='green', s=100, label="Start")
        plt.scatter(*locs[-2], color='orange', s=100, label="End")
        plt.legend()
        return plt

In [3]:
#
#
# select from crossover_values and mutation_values
#
#
#

# create the desired traveling salesman problem instace:
TSP_NAME = "d200-29"  # name of problem
tsp = TravelingSalesmanProblem(TSP_NAME)

crossover_values = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
mutation_values = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]

POPULATION_SIZE = 300
MAX_GENERATIONS = 200
HALL_OF_FAME_SIZE = 30


In [5]:
toolbox = base.Toolbox()

# define a single objective, minimizing fitness strategy:
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))

# create the Individual class based on list of integers:
creator.create("Individual", array.array, typecode='i', fitness=creator.FitnessMin)

# create an operator that generates randomly shuffled indices:
toolbox.register("randomOrder", random.sample, range(len(tsp)), len(tsp))

# create the individual creation operator to fill up an Individual instance with shuffled indices:
toolbox.register("individualCreator", tools.initIterate, creator.Individual, toolbox.randomOrder)

# create the population creation operator to generate a list of individuals:
toolbox.register("populationCreator", tools.initRepeat, list, toolbox.individualCreator)


# fitness calculation - compute the total distance of the list of cities represented by indices:
def tpsDistance(individual):
    return tsp.getTotalDistance(individual),  # return a tuple


toolbox.register("evaluate", tpsDistance)


# Genetic operators:
toolbox.register("select", tools.selTournament, tournsize=2)
toolbox.register("mate", tools.cxOrdered)
toolbox.register("mutate", tools.mutShuffleIndexes, indpb=1.0/len(tsp))

In [ ]:

import itertools


def run_ga(p_crossover, p_mutation):
    results = []
    for run in range(10):

        # create initial population (generation 0):
        population = toolbox.populationCreator(n=POPULATION_SIZE)

        # prepare the statistics object:
        stats = tools.Statistics(lambda ind: ind.fitness.values)
        stats.register("min", np.min)
        stats.register("avg", np.mean)

        # Hall of Fame to store the best individual
        hof = tools.HallOfFame(1)

        # perform the Genetic Algorithm flow with hof feature added:
        population, logbook = algorithms.eaSimple(population, toolbox, cxpb=p_crossover, mutpb=p_mutation,
                                                ngen=MAX_GENERATIONS, stats=stats, halloffame=hof, verbose=False)

        # Extract the best individual (solution)
        # best_solution = hof[0]
        # worst_solution = hof[-1]
        # Extract the best and worst individuals (solutions)
        best_solution = tools.selBest(population, 1)[0]
        worst_solution = tools.selWorst(population, 1)[0]
        results.append(tsp.getTotalDistance(best_solution))
    
       
        
        
        # Call the plotData method with the best solution
        tsp.plotData(best_solution)
        plt.show()
        # Call the plotData method with the worst solution
        tsp.plotData(worst_solution)
        plt.show()
        # Call the plotData method with a random solution in the population
        tsp.plotData(population[2])
        
        # plot statistics:
        minFitnessValues, meanFitnessValues = logbook.select("min", "avg")
        plt.figure(2)
        sns.set_style("whitegrid")
        plt.plot(minFitnessValues, color='red')
        plt.plot(meanFitnessValues, color='green')
        plt.xlabel('Generation')
        plt.ylabel('Min / Average Fitness')
        plt.title('Run {} of Min and Average fitness over Generations'.format(run + 1))

        # show both plots:
        plt.show()
        plt.close()
        return results


if __name__ == "__main__":
    tuning_results = []
    for p_crossover, p_mutation in itertools.product(crossover_values, mutation_values):
        results = run_ga(p_crossover, p_mutation)
        avg_distance = sum(results) / len(results)
        tuning_results.append((p_crossover, p_mutation, avg_distance))
    best_combination = min(tuning_results, key=lambda x: x[2])
    print(f"Best combination: P_CROSSOVER = {best_combination[0]}, P_MUTATION = {best_combination[1]}, Avg Distance = {best_combination[2]}")

In [3]:
#
#
# select from population_values and max_generations_values
#
#
#

# create the desired traveling salesman problem instace:
TSP_NAME = "d200-29"  # name of problem
tsp = TravelingSalesmanProblem(TSP_NAME)



# Genetic Algorithm constants:

HALL_OF_FAME_SIZE = 30
P_CROSSOVER = 0.5 # probability for crossover
P_MUTATION = 0.2   # probability for mutating an individual

population_values = [50, 100, 150, 200, 250, 300, 350, 400, 450, 500]
max_generations_values = [50, 100, 150, 200, 250, 300, 350, 400, 450, 500]


In [4]:
toolbox = base.Toolbox()

# define a single objective, minimizing fitness strategy:
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))

# create the Individual class based on list of integers:
creator.create("Individual", array.array, typecode='i', fitness=creator.FitnessMin)

# create an operator that generates randomly shuffled indices:
toolbox.register("randomOrder", random.sample, range(len(tsp)), len(tsp))

# create the individual creation operator to fill up an Individual instance with shuffled indices:
toolbox.register("individualCreator", tools.initIterate, creator.Individual, toolbox.randomOrder)

# create the population creation operator to generate a list of individuals:
toolbox.register("populationCreator", tools.initRepeat, list, toolbox.individualCreator)


# fitness calculation - compute the total distance of the list of cities represented by indices:
def tpsDistance(individual):
    return tsp.getTotalDistance(individual),  # return a tuple


toolbox.register("evaluate", tpsDistance)


# Genetic operators:
toolbox.register("select", tools.selTournament, tournsize=2)
toolbox.register("mate", tools.cxOrdered)
toolbox.register("mutate", tools.mutShuffleIndexes, indpb=1.0/len(tsp))

In [ ]:
import itertools


def run_ga(p_population, p_max_generations):
    results = []
    for run in range(10):

        # create initial population (generation 0):
        population = toolbox.populationCreator(n=p_population)

        # prepare the statistics object:
        stats = tools.Statistics(lambda ind: ind.fitness.values)
        stats.register("min", np.min)
        stats.register("avg", np.mean)

        # Hall of Fame to store the best individual
        hof = tools.HallOfFame(1)

        # perform the Genetic Algorithm flow with hof feature added:
        population, logbook = algorithms.eaSimple(population, toolbox, cxpb=P_CROSSOVER, mutpb=P_MUTATION,
                                                ngen=p_max_generations, stats=stats, halloffame=hof, verbose=False)

        # Extract the best individual (solution)
        # best_solution = hof[0]
        # worst_solution = hof[-1]
        # Extract the best and worst individuals (solutions)
        best_solution = tools.selBest(population, 1)[0]
        worst_solution = tools.selWorst(population, 1)[0]
        results.append(tsp.getTotalDistance(best_solution))
    
        # Call the plotData method with the best solution
        # tsp.plotData(best_solution)
        # plt.show()
        # Call the plotData method with the worst solution
        # tsp.plotData(worst_solution)
        # plt.show()
        # Call the plotData method with a random solution in the population
        tsp.plotData(population[2])
        
        # plot statistics:
        minFitnessValues, meanFitnessValues = logbook.select("min", "avg")
        plt.figure(2)
        sns.set_style("whitegrid")
        plt.plot(minFitnessValues, color='red')
        plt.plot(meanFitnessValues, color='green')
        plt.xlabel('Generation')
        plt.ylabel('Min / Average Fitness')
        plt.title('Run {} of Min and Average fitness over Generations'.format(run + 1))

        # show both plots:
        plt.show()
        plt.close()
        return results


if __name__ == "__main__":
    tuning_results = []
    for p_population, p_max_generations in itertools.product(population_values, max_generations_values):
        results = run_ga(p_population, p_max_generations)
        avg_distance = sum(results) / len(results)
        tuning_results.append((p_population, p_max_generations, avg_distance))
    best_combination = min(tuning_results, key=lambda x: x[2])
    print(f"Best combination: p_population = {best_combination[0]}, p_max_generations = {best_combination[1]}, Avg Distance = {best_combination[2]}")

In [13]:
#
#
# select from hall_of_fame_values
#
#
#


# create the desired traveling salesman problem instace:
TSP_NAME = "d200-29"  # name of problem
tsp = TravelingSalesmanProblem(TSP_NAME)


# Genetic Algorithm constants:
POPULATION_SIZE = 500
MAX_GENERATIONS = 450
P_CROSSOVER = 0.5 # probability for crossover
P_MUTATION = 0.2   # probability for mutating an individual

# population_values = [50, 100, 150, 200, 250, 300, 350, 400, 450, 500]
# max_generations_values = [50, 100, 150, 200, 250, 300, 350, 400, 450, 500]
hall_of_fame_values = [5, 10, 15, 20, 25, 30, 35, 40, 45, 50,55,60,65,70,75,80,85,90,95,100]

In [ ]:
toolbox = base.Toolbox()

# define a single objective, minimizing fitness strategy:
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))

# create the Individual class based on list of integers:
creator.create("Individual", array.array, typecode='i', fitness=creator.FitnessMin)

# create an operator that generates randomly shuffled indices:
toolbox.register("randomOrder", random.sample, range(len(tsp)), len(tsp))

# create the individual creation operator to fill up an Individual instance with shuffled indices:
toolbox.register("individualCreator", tools.initIterate, creator.Individual, toolbox.randomOrder)

# create the population creation operator to generate a list of individuals:
toolbox.register("populationCreator", tools.initRepeat, list, toolbox.individualCreator)


# fitness calculation - compute the total distance of the list of cities represented by indices:
def tpsDistance(individual):
    return tsp.getTotalDistance(individual),  # return a tuple


toolbox.register("evaluate", tpsDistance)


# Genetic operators:
toolbox.register("select", tools.selTournament, tournsize=2)
toolbox.register("mate", tools.cxOrdered)
toolbox.register("mutate", tools.mutShuffleIndexes, indpb=1.0/len(tsp))

In [ ]:
import itertools


def run_ga(p_hall_of_fame):
    results = []
    for run in range(10):

        # create initial population (generation 0):
        population = toolbox.populationCreator(n=POPULATION_SIZE)

        # prepare the statistics object:
        stats = tools.Statistics(lambda ind: ind.fitness.values)
        stats.register("min", np.min)
        stats.register("avg", np.mean)

        # Hall of Fame to store the best individual
        hof = tools.HallOfFame(p_hall_of_fame)

        # perform the Genetic Algorithm flow with hof feature added:
        population, logbook = algorithms.eaSimple(population, toolbox, cxpb=P_CROSSOVER, mutpb=P_MUTATION,
                                                ngen=MAX_GENERATIONS, stats=stats, halloffame=hof, verbose=False)

        # Extract the best individual (solution)
        # best_solution = hof[0]
        # worst_solution = hof[-1]
        # Extract the best and worst individuals (solutions)
        best_solution = tools.selBest(population, 1)[0]
        worst_solution = tools.selWorst(population, 1)[0]
        results.append(tsp.getTotalDistance(best_solution))
    
        # Call the plotData method with the best solution
        # tsp.plotData(best_solution)
        # plt.show()
        # Call the plotData method with the worst solution
        # tsp.plotData(worst_solution)
        # plt.show()
        # Call the plotData method with a random solution in the population
        tsp.plotData(population[2])
        
        # plot statistics:
        minFitnessValues, meanFitnessValues = logbook.select("min", "avg")
        plt.figure(2)
        sns.set_style("whitegrid")
        plt.plot(minFitnessValues, color='red')
        plt.plot(meanFitnessValues, color='green')
        plt.xlabel('Generation')
        plt.ylabel('Min / Average Fitness')
        plt.title('Run {} of Min and Average fitness over Generations'.format(run + 1))

        # show both plots:
        plt.show()
        plt.close()
        return results


if __name__ == "__main__":
    tuning_results = []
    for p_hall_of_fame in hall_of_fame_values:
        results = run_ga(p_hall_of_fame)
        avg_distance = sum(results) / len(results)
        tuning_results.append((p_hall_of_fame, avg_distance))
    best_combination = min(tuning_results, key=lambda x: x[1])
    print(f"Best combination: p_hall_of_fame = {best_combination[0]}, Avg Distance = {best_combination[1]}")

In [22]:
#
#
#
# all the constants has been selected, now pick from selection_methods
#
#
#

# create the desired traveling salesman problem instace:
TSP_NAME = "d200-29"  # name of problem
tsp = TravelingSalesmanProblem(TSP_NAME)



# Genetic Algorithm constants:
POPULATION_SIZE = 500
MAX_GENERATIONS = 450
HALL_OF_FAME_SIZE = 75
P_CROSSOVER = 0.5 # probability for crossover
P_MUTATION = 0.2   # probability for mutating an individual


In [23]:
toolbox = base.Toolbox()

# define a single objective, minimizing fitness strategy:
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))

# create the Individual class based on list of integers:
creator.create("Individual", array.array, typecode='i', fitness=creator.FitnessMin)

base.Fitness.crowding_dist = 0

# create an operator that generates randomly shuffled indices:
toolbox.register("randomOrder", random.sample, range(len(tsp)), len(tsp))

# create the individual creation operator to fill up an Individual instance with shuffled indices:
toolbox.register("individualCreator", tools.initIterate, creator.Individual, toolbox.randomOrder)

# create the population creation operator to generate a list of individuals:
toolbox.register("populationCreator", tools.initRepeat, list, toolbox.individualCreator)


# fitness calculation - compute the total distance of the list of cities represented by indices:
def tpsDistance(individual):
    return tsp.getTotalDistance(individual),  # return a tuple


toolbox.register("evaluate", tpsDistance)


# Genetic operators:

selection_methods = [
    (tools.selTournament, {'tournsize': 2}),
    (tools.selRoulette, {}),
    (tools.selNSGA2, {}),
    # (tools.selNSGA3, {'ref_points': tools.uniform_reference_points(nobj=1, p=12)}),
    (tools.selSPEA2, {}),
    (tools.selRandom, {}),
    (tools.selBest, {}),
    (tools.selWorst, {}),
    (tools.selTournamentDCD, {}),
    (tools.selDoubleTournament, {'fitness_size': 2, 'parsimony_size': 1, 'fitness_first': True}),
    (tools.selStochasticUniversalSampling, {}),
    (tools.selLexicase, {}),
    (tools.selEpsilonLexicase, {'epsilon': 0.01}),
    (tools.selAutomaticEpsilonLexicase, {}),
]

toolbox.register("mate", tools.cxOrdered)
toolbox.register("mutate", tools.mutShuffleIndexes, indpb=1.0/len(tsp))

In [ ]:
import itertools


def run_ga(selection_method, selection_args):
    results = []
    for run in range(10):

        if selection_method == tools.selTournamentDCD:
            adjusted_population_size = (POPULATION_SIZE // 4) * 4
        else:
            adjusted_population_size = POPULATION_SIZE

        # create initial population (generation 0):
        population = toolbox.populationCreator(n=adjusted_population_size)

        # prepare the statistics object:
        stats = tools.Statistics(lambda ind: ind.fitness.values)
        stats.register("min", np.min)
        stats.register("avg", np.mean)

        # Hall of Fame to store the best individual
        hof = tools.HallOfFame(1)

        toolbox.register("select", selection_method, **selection_args)


        # perform the Genetic Algorithm flow with hof feature added:
        population, logbook = algorithms.eaSimple(population, toolbox, cxpb=P_CROSSOVER, mutpb=P_MUTATION,
                                                ngen=MAX_GENERATIONS, stats=stats, halloffame=hof, verbose=False)

        # Extract the best individual (solution)
        # best_solution = hof[0]
        # worst_solution = hof[-1]
        # Extract the best and worst individuals (solutions)
        best_solution = tools.selBest(population, 1)[0]
        results.append(tsp.getTotalDistance(best_solution))
    
        worst_solution = tools.selWorst(population, 1)[0]
        
        
        # Call the plotData method with the best solution
        # tsp.plotData(best_solution)
        # plt.show()
        # Call the plotData method with the worst solution
        # tsp.plotData(worst_solution)
        # plt.show()
        # Call the plotData method with a random solution in the population
        tsp.plotData(population[2])
        
        # plot statistics:
        minFitnessValues, meanFitnessValues = logbook.select("min", "avg")
        plt.figure(2)
        sns.set_style("whitegrid")
        plt.plot(minFitnessValues, color='red')
        plt.plot(meanFitnessValues, color='green')
        plt.xlabel('Generation')
        plt.ylabel('Min / Average Fitness')
        plt.title('Run {} of Min and Average fitness over Generations'.format(run + 1))

        # show both plots:
        plt.show()
        plt.close()
        return results


if __name__ == "__main__":
    selection_results = []
    for selection_method, selection_args in selection_methods:
        # toolbox.register("select", selection_method)
        results = run_ga(selection_method, selection_args)
        avg_fitness = sum(results) / len(results)
        selection_results.append((selection_method.__name__, avg_fitness))

    best_selection = min(selection_results, key=lambda x: x[1])
    print(f"Best selection method: {best_selection[0]}, Avg Fitness = {best_selection[1]}")


In [ ]:
#
#
#
# pick the fitness size
#
#
#


# create the desired traveling salesman problem instace:
TSP_NAME = "d200-29"  # name of problem
tsp = TravelingSalesmanProblem(TSP_NAME)


# Genetic Algorithm constants:
POPULATION_SIZE = 500
MAX_GENERATIONS = 450
HALL_OF_FAME_SIZE = 75
P_CROSSOVER = 0.5 # probability for crossover
P_MUTATION = 0.2   # probability for mutating an individual


In [5]:
toolbox = base.Toolbox()

# define a single objective, minimizing fitness strategy:
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))

# create the Individual class based on list of integers:
creator.create("Individual", array.array, typecode='i', fitness=creator.FitnessMin)

# create an operator that generates randomly shuffled indices:
toolbox.register("randomOrder", random.sample, range(len(tsp)), len(tsp))

# create the individual creation operator to fill up an Individual instance with shuffled indices:
toolbox.register("individualCreator", tools.initIterate, creator.Individual, toolbox.randomOrder)

# create the population creation operator to generate a list of individuals:
toolbox.register("populationCreator", tools.initRepeat, list, toolbox.individualCreator)


# fitness calculation - compute the total distance of the list of cities represented by indices:
def tpsDistance(individual):
    return tsp.getTotalDistance(individual),  # return a tuple


toolbox.register("evaluate", tpsDistance)


toolbox.register("mate", tools.cxOrdered)
toolbox.register("mutate", tools.mutShuffleIndexes, indpb=1.0/len(tsp))

In [ ]:
import itertools


def run_ga():
    results = []
    for run in range(10):

        # create initial population (generation 0):
        population = toolbox.populationCreator(n=POPULATION_SIZE)

        # prepare the statistics object:
        stats = tools.Statistics(lambda ind: ind.fitness.values)
        stats.register("min", np.min)
        stats.register("avg", np.mean)

        # Hall of Fame to store the best individual
        hof = tools.HallOfFame(HALL_OF_FAME_SIZE)

        # perform the Genetic Algorithm flow with hof feature added:
        population, logbook = algorithms.eaSimple(population, toolbox, cxpb=P_CROSSOVER, mutpb=P_MUTATION,
                                                ngen=MAX_GENERATIONS, stats=stats, halloffame=hof, verbose=False)

        # Extract the best individual (solution)
        # best_solution = hof[0]
        # worst_solution = hof[-1]
        # Extract the best and worst individuals (solutions)
        best_solution = tools.selBest(population, 1)[0]
        results.append(tsp.getTotalDistance(best_solution))
    
        worst_solution = tools.selWorst(population, 1)[0]
        
        
        # Call the plotData method with the best solution
        tsp.plotData(best_solution)
        plt.show()
        # Call the plotData method with the worst solution
        tsp.plotData(worst_solution)
        plt.show()
        # Call the plotData method with a random solution in the population
        tsp.plotData(population[2])
        
        # plot statistics:
        minFitnessValues, meanFitnessValues = logbook.select("min", "avg")
        plt.figure(2)
        sns.set_style("whitegrid")
        plt.plot(minFitnessValues, color='red')
        plt.plot(meanFitnessValues, color='green')
        plt.xlabel('Generation')
        plt.ylabel('Min / Average Fitness')
        plt.title('Run {} of Min and Average fitness over Generations'.format(run + 1))

        # show both plots:
        plt.show()
        plt.close()
        return results


if __name__ == "__main__":
    tuning_results = []
    fitness_sizes = [5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60, 65, 70, 75, 80, 85, 90, 95, 100]

    for fitness_size in fitness_sizes:
        arg = {'fitness_size': fitness_size, 'parsimony_size': 1, 'fitness_first': True}
        toolbox.register(f"select", tools.selDoubleTournament, **arg)
        results = run_ga()
        avg_distance = sum(results) / len(results)
        tuning_results.append((fitness_size, avg_distance))
    
    best_combination = min(tuning_results, key=lambda x: x[1])
    print(f"Best combination: fitness size = {best_combination[0]}, Avg Distance = {best_combination[1]}")
    



#
#
# final result
#
#
#

POPULATION_SIZE = 500
MAX_GENERATIONS = 450
HALL_OF_FAME_SIZE = 75
P_CROSSOVER = 0.5
P_MUTATION = 0.2
tools.selDoubleTournament, {'fitness_size': 5, 'parsimony_size': 1, 'fitness_first': True}